In [8]:
import os
from getpass import getpass

watsonx_api_key = "0i-_-6pigerNnnRaU8_oiybRZz_UxMQuBHpE_copxSdw"
os.environ["WATSONX_APIKEY"] = watsonx_api_key

watsonx_project_id = "b596c884-f867-4771-afcc-f9fd10dae1a4"
os.environ["WATSONX_PROJECT_ID"] = watsonx_project_id


In [9]:
from llama_index.llms.ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
rag_gen_parameters = {
GenTextParamsMetaNames.DECODING_METHOD: "sample",
GenTextParamsMetaNames.MIN_NEW_TOKENS: 150,
GenTextParamsMetaNames.TEMPERATURE: 0.5,
GenTextParamsMetaNames.TOP_K: 5,
GenTextParamsMetaNames.TOP_P: 0.7
}

In [10]:
watsonx_llm = WatsonxLLM(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    max_new_tokens=512,
    params=rag_gen_parameters,
)

2025-09-09 16:30:37,837 - INFO - Client successfully initialized
2025-09-09 16:30:38,718 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-09 16:30:38,755 - INFO - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'


In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.embed_model = HuggingFaceEmbedding(
 model_name="BAAI/bge-small-en-v1.5"
)

2025-09-09 16:30:38,778 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2025-09-09 16:30:42,352 - INFO - 1 prompt is loaded, with the key: query


In [12]:
from llama_index.readers.file import PyMuPDFReader
loader = PyMuPDFReader()
pdf_doc = loader.load(file_path="./docs/housetomato.pdf")

In [13]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
index = VectorStoreIndex.from_documents(
 pdf_doc, transformations=[splitter], embed_model=Settings.embed_model
)

In [14]:
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine(retriever)

NameError: name 'retriever' is not defined

In [ ]:
query_gen_prompt_str = (
 "あなたは、1 つの入力クエリに基づいて複数の検索クエリを生成する有能なアシスタントです。\n"
 "{num_queries}個の検索クエリを、1 行につき 1 つずつ生成してください。\n"
 "以下のクエリに関連する検索クエリを生成してください：\n\n"
 "クエリ: {query} \n"
 "検索クエリ: \n"
)

In [ ]:
from llama_index.core.retrievers import QueryFusionRetriever
# this sets the LLM for the rest of the application
Settings.llm = watsonx_llm
# get retrievers
from llama_index.retrievers.bm25 import BM25Retriever
## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=2)
## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
 docstore=index.docstore, similarity_top_k=2
)
retriever = QueryFusionRetriever(
 [vector_retriever, bm25_retriever],
 similarity_top_k=4,
 num_queries=4, # set this to 1 to disable query generation
 mode="reciprocal_rerank",
 use_async=False,
 verbose=False,
 query_gen_prompt = query_gen_prompt_str # we could override the query generation prompt here
)

In [ ]:
watsonx_llm = WatsonxLLM(
 model_id="ibm/granite-3-2-8b-instruct",
 url="https://us-south.ml.cloud.ibm.com",
 project_id=os.getenv("WATSONX_PROJECT_ID"),
 max_new_tokens=512,
 params=rag_gen_parameters,
)

In [ ]:
import gradio as gr
def chat_function(message, history):
    try:
        response_obj = query_engine.query(message)
        response_text = response_obj.response
    except Exception as e:
        response_text = f"エラーが発生しました: {e}"

    return response_text
demo = gr.ChatInterface(
    fn=chat_function,
    title="トマトマスター",
    theme="soft",
    examples=[
        "トマトを家庭で育てるにはどうすればよいですか？",
        "トマトの栽培に最適な気候や土壌条件は何ですか？"
    ],
    type='messages'
)
demo.launch(inline=True, share=False)